# Анализ зарплат в России
В проекте использованы открытые данные из официальных источников:
[Сайт Росстата](https://rosstat.gov.ru/)  
[Таблицы уровня инфляции в России](https://уровень-инфляции.рф/).  
Для анализа выбраны виды экономичесой деятельности:
- сельское хозяйство, охота и лесное хозяйство, (
- 
Проанализирована динамика уровня среднегодовых зарплат в разрезе по видам экономической деятельности за последние 30 лет в России.  
Дополнительно учтена инфляция для подсчета реальной динамики зарплат и изучена корреляция с динамикой реальных зарплат (с учетом инфляции) некоторых макроэкономических показателей страны в разрезе по годам.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Загрузка данных

In [7]:
df = pd.read_csv('data/inflation_salaries_data.csv', sep=';')

In [8]:
df

,year,inflation,all_economic,education,production,building,IT
0,1991,"1,604",NaN,NaN,NaN,NaN,NaN
1,1992,"25,0885",NaN,NaN,NaN,NaN,NaN
2,1993,"8,3987",NaN,NaN,NaN,NaN,NaN
3,1994,"2,1502",NaN,NaN,NaN,NaN,NaN
4,1995,"1,3133",NaN,NaN,NaN,NaN,NaN
5,1996,"0,2181",NaN,NaN,NaN,NaN,NaN
6,1997,"0,1103",NaN,NaN,NaN,NaN,NaN
7,1998,"0,8444",NaN,NaN,NaN,NaN,NaN
8,1999,"0,3656",NaN,NaN,NaN,NaN,NaN
9,2000,"0,202","2223,4","1240,2","2365,2","2639,8",NaN
